<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study1/blob/main/marchine2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras

os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
# loading img
import PIL.Image as Image
p = '/content/drive/MyDrive/deajeon_motor'
os.chdir(p)

In [ ]:
def resize_img(img, size):
    return img.resize(size)

In [ ]:
def load_img(file_path):
    data = []
    print(p + file_path[1:] + '/')
    for f in os.listdir(file_path):
        data.append(resize_img(Image.open(p + file_path[1:] + '/' + f), (64, 64)))
    return data

In [ ]:
train_normal = load_img('./train/normal')  # img_data_list / element is image not nparray
test_normal = load_img('./test/normal')
val_normal = load_img('./validation/normal')

train_fault = load_img('./train/fault')
test_fault = load_img('./test/fault')
val_fault = load_img('./validation/fault')

/content/drive/MyDrive/deajeon_motor/train/normal/
/content/drive/MyDrive/deajeon_motor/test/normal/
/content/drive/MyDrive/deajeon_motor/validation/normal/
/content/drive/MyDrive/deajeon_motor/train/fault/
/content/drive/MyDrive/deajeon_motor/test/fault/
/content/drive/MyDrive/deajeon_motor/validation/fault/


In [ ]:
len(train_normal), len(test_normal), len(val_normal), len(train_fault), len(test_fault), len(val_fault)

(3000, 3000, 1500, 3000, 3000, 1500)

In [ ]:
# Convert images to arrays and define labels
def img_to_array(img):
    return np.array(img, dtype='float32') / 255.0

In [ ]:
train_normal_arr, train_normal_sol = np.array([img_to_array(normal) for normal in train_normal]), np.array([1] * len(train_normal))
test_normal_arr, test_normal_sol = np.array([img_to_array(normal) for normal in test_normal]), np.array([1] * len(test_normal))
val_normal_arr, val_normal_sol = np.array([img_to_array(normal) for normal in val_normal]), np.array([1] * len(val_normal))

train_fault_arr, train_fault_sol = np.array([img_to_array(fault) for fault in train_fault]), np.array([0] * len(train_fault))
test_fault_arr, test_fault_sol = np.array([img_to_array(fault) for fault in test_fault]), np.array([0] * len(test_fault))
val_fault_arr, val_fault_sol = np.array([img_to_array(fault) for fault in val_fault]), np.array([0] * len(val_fault))

In [ ]:
# Combine normal and fault data
train_img, train_sol = np.concatenate((train_normal_arr, train_fault_arr)), np.concatenate((train_normal_sol, train_fault_sol))
test_img, test_sol = np.concatenate((test_normal_arr, test_fault_arr)), np.concatenate((test_normal_sol, test_fault_sol))
val_img, val_sol = np.concatenate((val_normal_arr, val_fault_arr)), np.concatenate((val_normal_sol, val_fault_sol))

In [ ]:

# # Define and compile the model
# model = keras.models.Sequential()
# model.add(keras.layers.BatchNormalization(input_shape=train_img.shape[1:]))
# model.add(keras.layers.Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='elu', input_shape=(64, 64, 3)))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(keras.layers.Dropout(0.25))

# model.add(keras.layers.BatchNormalization(input_shape=train_img.shape[1:]))
# model.add(keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(keras.layers.Dropout(0.25))

# model.add(keras.layers.BatchNormalization(input_shape=train_img.shape[1:]))
# model.add(keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(keras.layers.Dropout(0.25))

# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256, activation='elu'))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Dense(128, activation='elu'))
# model.add(keras.layers.Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

model = Sequential()

# First block
model.add(BatchNormalization(input_shape=train_img.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

# Second block
model.add(BatchNormalization())
model.add(Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third block
model.add(BatchNormalization())
model.add(Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(256, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='elu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x=train_img, y=train_sol, batch_size=128, epochs=30, verbose=1, validation_data=(val_img, val_sol))

NameError: name 'model' is not defined

In [ ]:
# Evaluate the model on test data
model.evaluate(test_img, test_sol, batch_size=128)